In [ ]:
# 安装必要的依赖包
%pip install anthropic python-dotenv httpx mcp --quiet

print("✅ 依赖包安装完成！")


In [ ]:
# 导入必要的库并配置环境
import os
import json
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Any
import anthropic
import httpx
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 配置 Claude 客户端（支持代理）
client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    http_client=httpx.Client(proxy="http://127.0.0.1:7890/") if os.getenv("USE_PROXY") else None
)

print("🔧 环境配置完成！")
print(f"📅 当前时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# 定义辅助函数用于调用Claude API（流式输出）
def call_claude(prompt, system="", max_tokens=1000, temperature=0.7):
    """
    调用Claude API的辅助函数（流式输出）
    
    参数:
    - prompt: 用户输入的提示词
    - system: 系统提示词
    - max_tokens: 最大token数
    - temperature: 随机性参数（0-1，越高越随机）
    """
    full_response = ""
    
    with client.messages.stream(
        model="claude-3-5-sonnet-20241022",
        max_tokens=max_tokens,
        temperature=temperature,
        system=system,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    ) as stream:
        for text in stream.text_stream:
            print(text, end="", flush=True)
            full_response += text
    
    print()  # 添加换行
    return full_response

print("✅ Claude API 调用函数已配置！")


In [ ]:
# 模拟财经新闻数据库
financial_news_db = [
    {
        "id": "news_001",
        "title": "央行降准0.5个百分点 释放长期资金约1万亿元",
        "content": "中国人民银行决定于2024年2月5日下调金融机构存款准备金率0.5个百分点，预计释放长期资金约1万亿元。此次降准旨在保持流动性合理充裕，支持实体经济发展。",
        "source": "央行官网",
        "publish_time": "2024-02-05 16:00:00",
        "category": "货币政策",
        "keywords": ["央行", "降准", "流动性", "实体经济"],
        "sentiment": "positive",
        "importance": "high"
    },
    {
        "id": "news_002", 
        "title": "银行业净利润同比增长3.2% 资产质量持续改善",
        "content": "银保监会发布数据显示，2024年一季度银行业金融机构净利润同比增长3.2%，不良贷款率下降至1.67%。大型银行继续发挥头雁作用，中小银行经营稳健。",
        "source": "银保监会",
        "publish_time": "2024-01-20 10:30:00", 
        "category": "银行业绩",
        "keywords": ["银行", "净利润", "资产质量", "不良贷款"],
        "sentiment": "positive",
        "importance": "medium"
    },
    {
        "id": "news_003",
        "title": "人民币汇率双向波动 跨境资金流动总体均衡",
        "content": "外汇局表示，当前人民币汇率在合理均衡水平上保持基本稳定，跨境资金流动总体均衡。企业汇率避险意识不断增强，外汇市场运行更加成熟。",
        "source": "国家外汇管理局",
        "publish_time": "2024-01-15 14:20:00",
        "category": "汇率外汇", 
        "keywords": ["人民币", "汇率", "跨境资金", "外汇市场"],
        "sentiment": "neutral",
        "importance": "medium"
    }
]

print("📊 财经新闻数据库初始化完成")
print(f"📰 共加载 {len(financial_news_db)} 条新闻")


In [ ]:
# 实现资讯摘要 MCP Server 核心功能
class NewsAnalysisMCPServer:
    """银行资讯摘要 MCP 服务器"""
    
    def __init__(self):
        self.name = "bank-news-analysis"
        self.version = "1.0.0"
        self.capabilities = {
            "tools": ["search_news", "analyze_sentiment", "generate_summary"],
            "resources": ["news_database"],
            "prompts": ["morning_briefing"]
        }
    
    def search_news(self, category: str = None, keywords: List[str] = None, limit: int = 5) -> List[Dict]:
        """搜索财经新闻"""
        results = financial_news_db.copy()
        
        # 按类别筛选
        if category:
            results = [news for news in results if news["category"] == category]
        
        # 按关键词筛选
        if keywords:
            filtered_results = []
            for news in results:
                for keyword in keywords:
                    if keyword in news["title"] or keyword in news["content"] or keyword in news["keywords"]:
                        filtered_results.append(news)
                        break
            results = filtered_results
        
        # 按重要性和时间排序
        results.sort(key=lambda x: (x["importance"] == "high", x["publish_time"]), reverse=True)
        
        return results[:limit]
    
    def analyze_sentiment(self, news_id: str) -> Dict:
        """分析新闻情感倾向"""
        news = next((n for n in financial_news_db if n["id"] == news_id), None)
        if not news:
            return {"error": "新闻未找到"}
        
        return {
            "news_id": news_id,
            "sentiment": news["sentiment"],
            "confidence": 0.85,
            "impact_analysis": {
                "market_impact": "positive" if news["sentiment"] == "positive" else "neutral",
                "policy_relevance": "high" if "央行" in news["keywords"] else "medium"
            }
        }
    
    def generate_summary(self, news_list: List[str]) -> str:
        """生成新闻摘要"""
        if not news_list:
            return "未提供新闻内容"
        
        # 获取指定新闻
        selected_news = [n for n in financial_news_db if n["id"] in news_list]
        
        summary_prompt = f"""
        作为银行业资深分析师，请对以下财经新闻进行专业摘要分析：

        新闻内容：
        {json.dumps(selected_news, ensure_ascii=False, indent=2)}

        请按以下结构提供分析：
        1. 核心要点摘要（3-5个要点）
        2. 对银行业影响分析
        3. 政策解读与预期
        4. 投资建议（含风险提示）

        要求：专业、客观、简洁
        """
        
        return call_claude(
            prompt=summary_prompt,
            system="你是一位银行业资深分析师，专门负责财经资讯分析和投资建议。",
            max_tokens=800,
            temperature=0.3
        )

# 初始化服务器实例
news_server = NewsAnalysisMCPServer()
print("🚀 资讯摘要 MCP Server 初始化完成！")
print(f"📋 服务器名称：{news_server.name}")
print(f"🔧 支持功能：{', '.join(news_server.capabilities['tools'])}")


In [ ]:
# 测试 1: 搜索央行相关新闻
print("🔍 测试1: 搜索央行相关新闻")
print("=" * 50)

central_bank_news = news_server.search_news(keywords=["央行"], limit=2)
for news in central_bank_news:
    print(f"📰 {news['title']}")
    print(f"📅 {news['publish_time']} | 📊 重要性: {news['importance']}")
    print(f"🏷️ 关键词: {', '.join(news['keywords'])}")
    print()

print("✅ 搜索功能测试完成！")


In [ ]:
# 测试 2: 情感分析
print("🎭 测试2: 新闻情感分析")
print("=" * 50)

sentiment_result = news_server.analyze_sentiment("news_001")
print("分析结果：")
print(json.dumps(sentiment_result, ensure_ascii=False, indent=2))

print("\n✅ 情感分析功能测试完成！")


In [ ]:
# 测试 3: 生成智能摘要（这里将展示流式输出）
print("📝 测试3: 生成智能摘要")
print("=" * 50)

# 选择前两条新闻进行摘要
selected_news_ids = ["news_001", "news_002"]
print(f"正在分析新闻: {selected_news_ids}")
print("\n📊 AI分析结果：")
print("-" * 30)

# 调用摘要生成（会显示流式输出）
summary = news_server.generate_summary(selected_news_ids)

print("-" * 30)
print("✅ 智能摘要生成完成！")


In [ ]:
# 模拟股票行情数据
import random
from datetime import datetime, timedelta

def generate_stock_data(symbol: str, days: int = 30):
    """生成模拟股票数据"""
    base_price = 15.0 if symbol == "000001.SZ" else 12.0  # 平安银行 vs 招商银行
    data = []
    
    for i in range(days):
        date = (datetime.now() - timedelta(days=days-i)).strftime("%Y-%m-%d")
        
        # 模拟价格波动
        change_pct = random.uniform(-0.03, 0.03)  # -3% 到 +3%
        price = base_price * (1 + change_pct * (i + 1) / days)
        
        volume = random.randint(8000000, 15000000)  # 800万到1500万股
        
        data.append({
            "date": date,
            "symbol": symbol,
            "open": round(price * random.uniform(0.98, 1.02), 2),
            "high": round(price * random.uniform(1.00, 1.05), 2),
            "low": round(price * random.uniform(0.95, 1.00), 2),
            "close": round(price, 2),
            "volume": volume,
            "amount": round(volume * price, 2)
        })
    
    return data

# 生成银行股票数据
bank_stocks = {
    "000001.SZ": {
        "name": "平安银行",
        "industry": "银行",
        "data": generate_stock_data("000001.SZ", 30)
    },
    "600036.SH": {
        "name": "招商银行", 
        "industry": "银行",
        "data": generate_stock_data("600036.SH", 30)
    }
}

print("📊 银行股票数据生成完成")
for symbol, info in bank_stocks.items():
    latest = info["data"][-1]
    print(f"🏦 {info['name']} ({symbol}): ¥{latest['close']} (成交量: {latest['volume']:,})")


In [ ]:
# 实现交易趋势分析 MCP Server
class TradingAnalysisMCPServer:
    """银行股票交易趋势分析 MCP 服务器"""
    
    def __init__(self):
        self.name = "bank-trading-analysis"
        self.version = "1.0.0"
        self.capabilities = {
            "tools": ["get_stock_price", "calculate_indicators", "analyze_trend", "generate_signals"],
            "resources": ["market_data"],
            "prompts": ["trading_alert"]
        }
    
    def get_stock_price(self, symbol: str, days: int = 5) -> Dict:
        """获取股票价格数据"""
        if symbol not in bank_stocks:
            return {"error": f"股票代码 {symbol} 不存在"}
        
        stock_info = bank_stocks[symbol]
        recent_data = stock_info["data"][-days:]
        
        return {
            "symbol": symbol,
            "name": stock_info["name"],
            "industry": stock_info["industry"],
            "data": recent_data,
            "current_price": recent_data[-1]["close"],
            "change": round(recent_data[-1]["close"] - recent_data[-2]["close"], 2) if len(recent_data) > 1 else 0
        }
    
    def calculate_indicators(self, symbol: str, period: int = 14) -> Dict:
        """计算技术指标"""
        if symbol not in bank_stocks:
            return {"error": f"股票代码 {symbol} 不存在"}
        
        data = bank_stocks[symbol]["data"]
        prices = [d["close"] for d in data]
        volumes = [d["volume"] for d in data]
        
        # 简单移动平均线 (SMA)
        sma_5 = sum(prices[-5:]) / 5 if len(prices) >= 5 else prices[-1]
        sma_20 = sum(prices[-20:]) / 20 if len(prices) >= 20 else sum(prices) / len(prices)
        
        # 相对强弱指标 (RSI) - 简化计算
        if len(prices) >= period:
            gains = [max(prices[i] - prices[i-1], 0) for i in range(1, len(prices))]
            losses = [max(prices[i-1] - prices[i], 0) for i in range(1, len(prices))]
            avg_gain = sum(gains[-period:]) / period
            avg_loss = sum(losses[-period:]) / period
            rsi = 100 - (100 / (1 + avg_gain / (avg_loss + 0.001)))
        else:
            rsi = 50  # 中性值
        
        # 成交量指标
        avg_volume = sum(volumes[-5:]) / 5 if len(volumes) >= 5 else volumes[-1]
        volume_ratio = volumes[-1] / avg_volume if avg_volume > 0 else 1.0
        
        return {
            "symbol": symbol,
            "indicators": {
                "SMA_5": round(sma_5, 2),
                "SMA_20": round(sma_20, 2),
                "RSI": round(rsi, 2),
                "volume_ratio": round(volume_ratio, 2),
                "current_price": prices[-1]
            },
            "signals": {
                "trend": "上涨" if sma_5 > sma_20 else "下跌",
                "rsi_signal": "超买" if rsi > 70 else "超卖" if rsi < 30 else "中性",
                "volume_signal": "放量" if volume_ratio > 1.5 else "缩量" if volume_ratio < 0.7 else "正常"
            }
        }
    
    def analyze_trend(self, symbol: str) -> str:
        """分析股票趋势"""
        indicators = self.calculate_indicators(symbol)
        if "error" in indicators:
            return indicators["error"]
        
        stock_info = bank_stocks[symbol]
        data = stock_info["data"][-7:]  # 最近7天数据
        
        trend_prompt = f"""
        作为资深量化分析师，请分析以下银行股票的交易趋势：

        股票信息：
        - 代码：{symbol}
        - 名称：{stock_info["name"]}
        - 当前价格：¥{indicators["indicators"]["current_price"]}

        技术指标：
        - 5日均线：¥{indicators["indicators"]["SMA_5"]}
        - 20日均线：¥{indicators["indicators"]["SMA_20"]} 
        - RSI：{indicators["indicators"]["RSI"]}
        - 成交量比：{indicators["indicators"]["volume_ratio"]}

        信号分析：
        - 趋势信号：{indicators["signals"]["trend"]}
        - RSI信号：{indicators["signals"]["rsi_signal"]}
        - 成交量信号：{indicators["signals"]["volume_signal"]}

        最近7天价格走势：
        {json.dumps(data, ensure_ascii=False, indent=2)}

        请提供：
        1. 趋势判断（短期/中期）
        2. 关键阻力位和支撑位
        3. 交易建议（含风险控制）
        4. 后市预期

        注意：必须包含风险提示。
        """
        
        return call_claude(
            prompt=trend_prompt,
            system="你是一位银行股票资深量化分析师，专注于技术分析和风险控制。",
            max_tokens=800,
            temperature=0.3
        )

# 初始化交易分析服务器
trading_server = TradingAnalysisMCPServer()
print("🚀 交易趋势分析 MCP Server 初始化完成！")
print(f"📋 服务器名称：{trading_server.name}")
print(f"🔧 支持功能：{', '.join(trading_server.capabilities['tools'])}")


In [ ]:
# 测试 1: 获取股票价格
print("💰 测试1: 获取平安银行股价")
print("=" * 50)

price_data = trading_server.get_stock_price("000001.SZ", days=3)
print(f"📊 {price_data['name']} ({price_data['symbol']})")
print(f"💵 当前价格: ¥{price_data['current_price']}")
print(f"📈 涨跌: ¥{price_data['change']}")

print("\n最近3天数据:")
for day_data in price_data['data']:
    print(f"📅 {day_data['date']}: 收盘¥{day_data['close']} | 成交量{day_data['volume']:,}")

print("\n✅ 价格获取功能测试完成！")


In [ ]:
# 测试 2: 计算技术指标
print("📊 测试2: 技术指标计算")
print("=" * 50)

indicators = trading_server.calculate_indicators("000001.SZ")
print("技术指标结果:")
print(json.dumps(indicators, ensure_ascii=False, indent=2))

print("\n✅ 技术指标计算功能测试完成！")


In [ ]:
# 测试 3: 趋势分析（流式输出演示）
print("🔍 测试3: 智能趋势分析")
print("=" * 50)

print("正在分析平安银行交易趋势...")
print("\n📊 AI趋势分析结果：")
print("-" * 30)

# 调用趋势分析（会显示流式输出）
trend_analysis = trading_server.analyze_trend("000001.SZ")

print("-" * 30)
print("✅ 趋势分析完成！")

# 添加免责声明
print("\n⚠️ 免责声明：以上分析仅供参考，不构成投资建议。投资有风险，决策需谨慎。")


In [ ]:
# 演示银行级安全控制
class SecureMCPServer:
    """银行级安全的 MCP 服务器基类"""
    
    def __init__(self, server_name: str):
        self.server_name = server_name
        self.access_log = []
        self.allowed_operations = set()
        self.user_permissions = {}
    
    def log_access(self, user_id: str, operation: str, params: dict, result: str):
        """记录访问日志（合规要求）"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "server": self.server_name,
            "user_id": user_id,
            "operation": operation,
            "params": params,
            "result_status": result,
            "ip_address": "192.168.1.100"  # 模拟IP
        }
        self.access_log.append(log_entry)
        
    def check_permission(self, user_id: str, operation: str) -> bool:
        """检查用户权限"""
        user_perms = self.user_permissions.get(user_id, set())
        return operation in user_perms
    
    def require_approval(self, operation: str, user_id: str, params: dict) -> bool:
        """需要人工审批的操作"""
        high_risk_operations = ["transfer_funds", "modify_account", "send_alert"]
        
        if operation in high_risk_operations:
            print(f"🚨 高风险操作需要审批:")
            print(f"   用户: {user_id}")
            print(f"   操作: {operation}")
            print(f"   参数: {params}")
            
            # 模拟审批流程
            approval = input("   是否批准？(y/n): ").lower() == 'y'
            return approval
        
        return True  # 低风险操作自动通过

# 演示安全配置
secure_server = SecureMCPServer("bank-secure-server")

# 配置用户权限
secure_server.user_permissions = {
    "analyst001": {"search_news", "analyze_sentiment", "get_stock_price"},
    "trader002": {"search_news", "get_stock_price", "calculate_indicators"},
    "manager003": {"search_news", "analyze_sentiment", "get_stock_price", "send_alert"}
}

print("🔐 银行级安全控制演示")
print("=" * 50)

# 测试权限检查
test_user = "analyst001"
test_operation = "search_news"

if secure_server.check_permission(test_user, test_operation):
    print(f"✅ 用户 {test_user} 有权限执行 {test_operation}")
    secure_server.log_access(test_user, test_operation, {"query": "央行政策"}, "success")
else:
    print(f"❌ 用户 {test_user} 无权限执行 {test_operation}")

# 显示访问日志
print("\n📋 访问日志:")
for log in secure_server.access_log:
    print(f"   {log['timestamp']}: {log['user_id']} -> {log['operation']} ({log['result_status']})")

print("\n🛡️ 安全控制功能演示完成！")
